In [ ]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2 MB 31 kB/s 
     |████████████████████████████████| 198 kB 60.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=cb772fac0d4016d5c8114d41c92c71a950f31975431d0f3a30a0d69660568f0e
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('spotify_classification')\
        .getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = spark.read.option("header", "true").csv("drive/My Drive/spot_dataset.csv")

In [ ]:
df.show(10)

+---+--------------------+----------------+-----------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+---------------------+
|_c0|            id_track|popularity_track|duration_ms| collect_list_genres|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|sum_artist_followers|sum_artist_popularity|
+---+--------------------+----------------+-----------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+---------------------+
|  0|007dUtG6y7FlTx9Ss...|               0|    1695512|[[''], ['drama', ...|  1938-07-03|       0.607| 0.484|  2| -12.234|   0|      0.959|        0.99|         2.7e-05|   0.384|  0.453| 69.539|             4|                5877|           

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
df_cla = df.drop("_c0","id_track","collect_list_genres","release_date")

In [ ]:
df_cla.printSchema()

root
 |-- popularity_track: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- sum_artist_followers: string (nullable = true)
 |-- sum_artist_popularity: string (nullable = true)



In [ ]:
df_cla = df_cla.withColumn("popularity_track", df_cla.popularity_track.cast(FloatType())) \
         .withColumn("duration_ms", df_cla.duration_ms.cast(FloatType())) \
         .withColumn("danceability", df_cla.danceability.cast(FloatType())) \
         .withColumn("energy", df_cla.energy.cast(FloatType())) \
         .withColumn("key", df_cla.key.cast(FloatType())) \
         .withColumn("loudness", df_cla.loudness.cast(FloatType())) \
         .withColumn("mode", df_cla.mode.cast(FloatType())) \
         .withColumn("speechiness", df_cla.speechiness.cast(FloatType())) \
         .withColumn("acousticness", df_cla.acousticness.cast(FloatType())) \
         .withColumn("instrumentalness", df_cla.instrumentalness.cast(FloatType())) \
         .withColumn("liveness", df_cla.liveness.cast(FloatType())) \
         .withColumn("valence", df_cla.valence.cast(FloatType())) \
         .withColumn("tempo", df_cla.tempo.cast(FloatType())) \
         .withColumn("time_signature", df_cla.time_signature.cast(FloatType())) \
         .withColumn("sum_artist_followers", df_cla.sum_artist_followers.cast(FloatType())) \
         .withColumn("sum_artist_popularity", df_cla.sum_artist_popularity.cast(FloatType())) \

In [ ]:
df_cla.printSchema()

root
 |-- popularity_track: float (nullable = true)
 |-- duration_ms: float (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: float (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: float (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- time_signature: float (nullable = true)
 |-- sum_artist_followers: float (nullable = true)
 |-- sum_artist_popularity: float (nullable = true)



In [50]:
df_cla.select(df_cla.popularity_track).collect()

[Row(popularity_track=0.0),
 Row(popularity_track=58.0),
 Row(popularity_track=7.0),
 Row(popularity_track=21.0),
 Row(popularity_track=12.0),
 Row(popularity_track=10.0),
 Row(popularity_track=34.0),
 Row(popularity_track=0.0),
 Row(popularity_track=47.0),
 Row(popularity_track=37.0),
 Row(popularity_track=30.0),
 Row(popularity_track=38.0),
 Row(popularity_track=13.0),
 Row(popularity_track=5.0),
 Row(popularity_track=59.0),
 Row(popularity_track=18.0),
 Row(popularity_track=49.0),
 Row(popularity_track=50.0),
 Row(popularity_track=31.0),
 Row(popularity_track=14.0),
 Row(popularity_track=30.0),
 Row(popularity_track=25.0),
 Row(popularity_track=41.0),
 Row(popularity_track=45.0),
 Row(popularity_track=41.0),
 Row(popularity_track=23.0),
 Row(popularity_track=0.0),
 Row(popularity_track=36.0),
 Row(popularity_track=24.0),
 Row(popularity_track=43.0),
 Row(popularity_track=2.0),
 Row(popularity_track=0.0),
 Row(popularity_track=1.0),
 Row(popularity_track=11.0),
 Row(popularity_track=

In [51]:
df_cla.select(max(df_cla.popularity_track)).collect()

[Row(max(popularity_track)=100.0)]

In [83]:
from pyspark.ml.feature import VectorAssembler

In [84]:
assembler = VectorAssembler( 
inputCols=['popularity_track',
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'sum_artist_followers',
 'sum_artist_popularity'], 
outputCol="features")
output=assembler.transform(df_cla)

In [61]:
final_data = output.select("features", df_cla.popularity_track)

In [62]:
train, test = final_data.randomSplit([0.7, 0.3])

In [66]:
lr = LogisticRegression(labelCol ="target", featuresCol ="features")

In [69]:
train.show(10)

Py4JJavaError: ignored

In [68]:
model=lr.fit(train)
predict_train = model.transform(train)
predict_test  = model.transform(test)
predict_test.select("target", "prediction").show(10)

IllegalArgumentException: ignored

In [85]:
from pyspark.ml.feature import Bucketizer
bucketBorders=[0,0.2,0.4,0.6,0.8,1]

In [93]:
bucketer=Bucketizer().setSplits(bucketBorders).setInputCol("popularity_track").setOutputCol("Buckets")
df_buck = bucketer.transform(df_cla)

In [96]:
df_buck.dtypes

[('popularity_track', 'float'),
 ('duration_ms', 'float'),
 ('danceability', 'float'),
 ('energy', 'float'),
 ('key', 'float'),
 ('loudness', 'float'),
 ('mode', 'float'),
 ('speechiness', 'float'),
 ('acousticness', 'float'),
 ('instrumentalness', 'float'),
 ('liveness', 'float'),
 ('valence', 'float'),
 ('tempo', 'float'),
 ('time_signature', 'float'),
 ('sum_artist_followers', 'float'),
 ('sum_artist_popularity', 'float'),
 ('Buckets', 'double')]

In [97]:
df_buck.show(5)

Py4JJavaError: ignored